In [1]:
import numpy as np
import cPickle as pickle
import pandas as pd


In [51]:
meanSentiment = "../Final_features/median_sentiments.csv"
aesthetic_csv = "../Final_features/total_aesthetic.pk"
audiopop_csv = "../Final_features/popAudio_features.csv"
audioUnpop_csv = "../Final_features/unpopAudioFeatures.csv"
socialFeatures = "../Final_features/postSocialMetaFeatures.pk"

In [4]:
f = open(aesthetic_csv , "r+")
aesFeats = pickle.load(f)
f.close()


['1282183367848497152' '-1.10801774134e-14' '8.58375272881'
 '0.0526322079498' '0.0009765625' '0.00244140625' '0.540988032549'
 '0.446194058642' '0.850565751465' '0.0610475900012' '0.225965286793' '0.0'
 '0.0' '0.00567806518798' '-0.463018808309' '0.0122138666405'
 '0.00352911656145' '0.53609375' '0.00407062926302' '1.0']


In [53]:
print aesFeats.shape

(18474, 20)


In [58]:
f = open(socialFeatures , "r+")
socialFeats = pickle.load(f)
f.close()
print socialFeats[104][0]

1294156026651774976


In [55]:
def readFeatureFile(color_features):
    with open(color_features) as g:
        featureLines = g.readlines()
    print len(featureLines)
    return featureLines


def readAudioFeatures(lines):
    featureDict = {}
    
    for line in lines:
        components = line.strip().split(',')
        postId = components[0].strip().split('.')[0]
        feats = [float(x) for x in components[1:]]
        cleanFeats = []
        for x in feats:
            if str(x) != 'nan':
                cleanFeats.append(float(x))
            else:
                cleanFeats.append(0.0)
        featureDict[postId] = cleanFeats
            
    return featureDict

def readSentiFeatures(lines):
    sentiDict = {}
    for line in lines:
        comp = line.split(',')
        sentiDict[comp[0]] = float(comp[1].strip())
    return sentiDict

def mergeAllFeatures(aesthetic , sentiment , audio , social):
    totalFeatures = []
    labelFeatures = []
    for aes in aesthetic:
        if aes[0] in sentiment:
            if aes[0] in audio:
                if aes[0] in social[:,0]:
                    i = social[0,:].index(aes[0])
                    featureList = aes[1:] + audio[aes[0]] + sentiment[aes[0]] + social[i,1:3]
                    totalFeatures.append(featureList)
                    labelFeatures.append(social[i,3:])
    return totalFeatures, labelFeatures

In [38]:
popAudio_csv = readFeatureFile(audiopop_csv)

11446


In [39]:
unpopAudio_csv = readFeatureFile(audioUnpop_csv)

11992


In [40]:
totalFeats = popAudio_csv + unpopAudio_csv
audioDict = readAudioFeatures(totalFeats)

In [41]:
ex =  audioDict['1381729172183040000']

In [43]:
print ex

[0.0, 0.16844, 40.915827, 0.10944, 0.7735, 0.333333]


In [45]:
senti_csv = readFeatureFile(meanSentiment)

23441


In [49]:
sentiDict = readSentiFeatures(senti_csv)

In [50]:
sentiDict['1381729172183040000']

3.6

In [56]:
trainFeats, labelFeats = mergeAllFeatures(aesFeats , sentiDict , audioDict , socialFeatures)

TypeError: string indices must be integers, not tuple